<center>

#  TOP Verkleg Æfing
### Snúningspendúll (Torsional Pendulum)

*Stefán Örn Lárusson*

*Torfi Þorgrímsson*

In [235]:
using RemoteREPL
if !isdefined(Base, :task)
	task = @async serve_repl()
end

Task (failed) @0x0000000075aa4200
IOError: listen: address already in use (EADDRINUSE)
Stacktrace:
 [1] uv_error
   @ .\libuv.jl:97 [inlined]
 [2] #listen#13
   @ C:\Users\stebb\AppData\Local\Programs\Julia-1.8.2\share\julia\stdlib\v1.8\Sockets\src\Sockets.jl:629 [inlined]
 [3] #listen#10
   @ C:\Users\stebb\AppData\Local\Programs\Julia-1.8.2\share\julia\stdlib\v1.8\Sockets\src\Sockets.jl:622 [inlined]
 [4] #listen#12
   @ C:\Users\stebb\AppData\Local\Programs\Julia-1.8.2\share\julia\stdlib\v1.8\Sockets\src\Sockets.jl:626 [inlined]
 [5] listen
   @ C:\Users\stebb\AppData\Local\Programs\Julia-1.8.2\share\julia\stdlib\v1.8\Sockets\src\Sockets.jl:626 [inlined]
 [6] serve_repl(address::Sockets.IPv4, port::Int64; kws::Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}})
   @ RemoteREPL C:\Users\stebb\.julia\packages\RemoteREPL\tVFcu\src\server.jl:241
 [7] serve_repl (repeats 2 times)
   @ C:\Users\stebb\.julia\packages\RemoteREPL\tVFcu\src\server.jl:240 [inlined]
 [8] (::var"#43#4

In [236]:
using DataFrames, Statistics, CSV
using Measurements, Unitful
using Plots, PlotThemes
using EasyFit, Peaks, Symbolics
using Latexify, LaTeXStrings
plotlyjs();
theme(:solarized_light)
val(x) = (Measurements.value∘ustrip∘upreferred)(x)
err(x) = (Measurements.uncertainty∘ustrip∘upreferred)(x)

err (generic function with 1 method)

## Gögn

### Gögn 1, mæling á kraft og θ

In [237]:
data = CSV.read("data.csv", DataFrame)

data = data[data[!,1] .> -0.15,:]

Δθ = 1u"°"
ΔF = 0.0125u"N"

data[!,2] = data[!,2]u"rad"

off = 13
plot(data[!,2][1:(end -end÷2 - off)],data[!,1][1:(end - end÷2 - off)]u"N",
	ribbon= ΔF,
	fillalpha = 0.3,
	xlabel = "Theta", 
	ylabel = "Force", 
	labels ="Fram")
plot!(data[!,2][(end -end÷2 - off):end],data[!,1][(end - end÷2 - off):end]u"N",
	ribbon= ΔF,
	fillalpha = 0.3,
	labels ="Til Baka",
	legend = :top)

In [238]:
savefig("data1.svg")

### Gögn 2, Mæling á θ yfir tíma með málmskífu
Með hápunktum og y-ás hliðraður

In [239]:
data2 = CSV.read("data2.csv", DataFrame)

data2[!,2] = data2[!,2] .- data2[end,2]

data2[!,2] = data2[!,2]u"rad"# .± 1u"°"
data2[!,1] = data2[!,1]u"s"

plot(data2[!,1], data2[!,2],
	xlabel = "Time",
	ylabel = "θ",
	labels = :none)

In [240]:
savefig("data2.svg")

#### Gögn 2 nema abs

In [241]:
plot(data2[!,1], abs.(data2[!,2]),
	xlabel = "Time [s]",
	ylab = "|θ| [rad]",
	labels = :none)

data2[!,2] = abs.(data2[!,2])

peaks = findmaxima(data2[!,2])
peaks = peaks[1][peaks[2] .< 13 .&& peaks[2].> .2]

scatter!(data2[peaks,1],data2[peaks,2],
		labels = "Peaks")

In [242]:
savefig("data2abspeaks.svg")

#### Gögn 2 hápunktar nema $\log_e$ skali á y-ás

In [243]:
plot(data2[peaks,1], log.(data2[peaks,2]),
	xlabel = "Time [s]",
	ylabel = "ln(θ) [rad]",
	labels = :none)

In [244]:
savefig("data2lntheta.svg")

### Gögn 3, Mæling á θ yfir tíma nema með segul á topp súlunar

In [245]:
data3 = CSV.read("data3.csv", DataFrame)

data3[!,2] = data3[!,2] .- data3[end,2]

data3[!,2] = data3[!,2]u"rad" #.± 1u"°"
data3[!,1] = data3[!,1]u"s"

plot(data3[!,1], data3[!,2],
	xlabel = "Time [s]",
	ylabel = "θ [rad]",
	labels = :none)

In [246]:
savefig("data3.svg")

### Gögn 4, Mæling á θ yfir tíma nema með segul á hlið topp súlunar

In [247]:
data4 = CSV.read("data4.csv", DataFrame)

data4[!,2] = data4[!,2] .- data4[end,2]

data4[!,2] = data4[!,2]u"rad" #.± 1u"°"
data4[!,1] = data4[!,1]u"s"

plot(data4[!,1], data4[!,2],
	xlabel = "Time [s]",
	ylabel = "θ [rad]",
	labels = :none)

In [248]:
savefig("data4.svg")

## Útreykningar

### Góða gamla fallið

In [249]:
"""
Fall til að fynna jöfnu óvissu fyrir gefna jöfnu
"""
function findErrorFromSym(symExpr; errorPrefix = "Err")
	vars = Symbolics.get_variables(symExpr)
	varErrs = []

	for i in vars
		push!(varErrs, Symbolics.variable(string(errorPrefix,i)))
	end

	Dvars = [expand_derivatives(Differential(i)(symExpr)) for i in vars]

	symErr = sqrt(sum((Dvars[i]*varErrs[i])^2 for i in eachindex(vars)))
	
	return symErr
end

findErrorFromSym

### Mælingar

In [250]:
rskvfull = (5.16±0.01)u"cm"
rskvinn = (0.27±0.01)u"cm"
(rₛ = rskvfull-rskvinn) |> latexify

L"$4.89 ± 0.014 cm$"

In [251]:
(kₛ = fitlinear(val.(data[!,2]),val.(data[!,1])).a*1u"N*m") |> latexify

L"$0.010578748135938463 m N$"

In [252]:
(r = (9.5±.1)u"cm") |> latexify

L"$9.5 ± 0.1 cm$"

In [253]:
(m = (122±1)u"g")|> latexify

L"$122.0 ± 1.0 g$"

### Jöfnur og útreikningar

#### Data 2

In [254]:
k = fitlinear(val.(data2[peaks,1]),log.(ℯ, val.(data2[peaks,2])))


 ------------------- Linear Fit ------------- 

 Equation: y = ax + b 

 With: a = -0.08148727018407854
       b = 3.052224788636173



 Pearson correlation coefficient, R = 0.977900930301875
 Average square residue = 0.033998889529418395

 Predicted Y: ypred = [2.793095269450803, 2.6056745480274226...
 residues = [0.24560513336103496, 0.1465986582739438...

 -------------------------------------------- 


##### Δk

In [255]:
latexify(:(Δk=sqrt(n/(n*sum(x.^2)-sum(x.^2)))),env = :eq)

L"\begin{equation}
{\Delta}k = \sqrt{\frac{n}{n \cdot \sum x^{2} - \sum x^{2}}}
\end{equation}
"

In [256]:
x = data2[peaks,1]
n = length(x)
(Δk = sqrt(n/(n*sum(val.(x).^2)-sum(val.(x).^2)))) |> latexify

L"$0.011224449816045548$"

##### b og Δb

In [257]:
@variables κ R M Δκ ΔR ΔM
b = -1/2 *M*R^2*κ

-0.5M*κ*(R^2)

In [258]:
Δb = findErrorFromSym(b,errorPrefix = "Δ")

sqrt(0.25(M^2)*(R^4)*(Δκ^2) + 0.25(R^4)*(ΔM^2)*(κ^2) + (M^2)*(R^2)*(ΔR^2)*(κ^2))

In [259]:
b = substitute(b,
	Dict([
		κ => k.a,
		R => val(r),
		M => val(m)
	]))
b = b

Δb = substitute(Δb, 
	Dict([
		κ => k.a, 
		M => val(m), 
		R => val(r), 
		Δκ => val(Δk), 
		ΔM => err(m), 
		ΔR => err(r)
	])
)

(b = (Float64(b.val) ± Float64(Δb.val))*u"kg * m^2 * s^-1") |> latexify

L"$4.49e-5 ± 6.3e-6 kg m^2 s^-1$"

In [260]:
barr = []
karr = []
push!(barr, b)
push!(karr, k.a±Δk)

1-element Vector{Any}:
 -0.081 ± 0.011

#### Data 3

In [261]:
data3[!,2] = abs.(data3[!,2])

peaks = findmaxima(data3[!,2])
peaks = peaks[1][peaks[2] .< 20 .&& peaks[2].> .1]

plot(data3[!,1], abs.(data3[!,2]),
	xlabel = "Time [s]",
	ylab = "|θ| [rad]",
	labels = :none)
scatter!(data3[peaks,1],data3[peaks,2],
labels = "Peaks")

In [262]:
plot(data3[peaks,1], log.(data3[peaks,2]),
	xlabel = "Time [s]",
	ylabel = "ln(θ) [rad]",
	labels = :none)

In [263]:
k = fitlinear(val.(data3[peaks,1]),log.(ℯ, data3[peaks,2]))


 ------------------- Linear Fit ------------- 

 Equation: y = ax + b 

 With: a = -0.07214480910790268
       b = 3.416950993909723

 Pearson correlation coefficient, R = 0.9773249618080894
 Average square residue = 0.04288776699844387

 Predicted Y: ypred = [3.2178313207719116, 3.0518982598237354...
 residues = [0.2804166125556393, 0.19437209293520752...

 -------------------------------------------- 


##### Δk

In [264]:
latexify(:(Δk=sqrt(n/(n*sum(x.^2)-sum(x.^2)))),env = :eq)

L"\begin{equation}
{\Delta}k = \sqrt{\frac{n}{n \cdot \sum x^{2} - \sum x^{2}}}
\end{equation}
"

In [265]:
x = val.(data3[peaks,1])
n = length(x)
(Δk = sqrt(n/(n*sum(x.^2)-sum(x.^2)))) |> latexify

L"$0.008204514174430747$"

##### b og Δb

In [266]:
@variables κ R M Δκ ΔR ΔM
b = -1/2 *M*R^2*κ

-0.5M*κ*(R^2)

In [267]:
Δb = findErrorFromSym(b,errorPrefix = "Δ")

sqrt(0.25(M^2)*(R^4)*(Δκ^2) + 0.25(R^4)*(ΔM^2)*(κ^2) + (M^2)*(R^2)*(ΔR^2)*(κ^2))

In [268]:
b = substitute(b,
	Dict([
		κ => k.a,
		R => val(r),
		M => val(m)
	]))
b = b.val

Δb = substitute(Δb, 
	Dict([
		κ => k.a, 
		M => val(m), 
		R => val(r), 
		Δκ => val(Δk), 
		ΔM => err(m), 
		ΔR => err(r)
	])
)
Δb = Δb.val

(b = (b ± Δb)*u"kg * m^2 * s^-1") |> latexify

L"$3.97e-5 ± 4.6e-6 kg m^2 s^-1$"

In [269]:
push!(barr, b)
push!(karr, k.a±Δk)

2-element Vector{Any}:
  -0.081 ± 0.011
 -0.0721 ± 0.0082

#### Data 4

In [270]:
data4[!,2] = abs.(data4[!,2])

peaks = findmaxima(data4[!,2])
peaks = peaks[1][peaks[2] .< 20 .&& peaks[2].> .1]

plot(data4[!,1], abs.(data4[!,2]),
	xlabel = "Time [s]",
	ylab = "|θ| [rad]",
	labels = :none)
scatter!(data4[peaks,1],data4[peaks,2],
labels = "Peaks")

In [271]:
plot(data4[peaks,1], log.(data4[peaks,2]),
	xlabel = "Time [s]",
	ylabel = "ln(θ) [rad]",
	labels = :none)

In [272]:
k = fitlinear(val.(data4[peaks,1]),log.(ℯ, data4[peaks,2]))


 ------------------- Linear Fit ------------- 

 Equation: y = ax + b 

 With: a = -0.07584442705747646
       b = 3.412884951395409

 Pearson correlation coefficient, R = 0.9751894548188891
 Average square residue = 0.04687406023528046



 Predicted Y: ypred = [3.1929361129287273, 3.0184939306965317...
 residues = [0.27137255245007363, 0.2187157655431049...

 -------------------------------------------- 


##### Δk

In [273]:
latexify(:(Δk=sqrt(n/(n*sum(x.^2)-sum(x.^2)))),env = :eq)

L"\begin{equation}
{\Delta}k = \sqrt{\frac{n}{n \cdot \sum x^{2} - \sum x^{2}}}
\end{equation}
"

In [274]:
x = val.(data4[peaks,1])
n = length(x)
(Δk = sqrt(n/(n*sum(x.^2)-sum(x.^2)))) |> latexify

L"$0.008810222225793729$"

##### b og Δb

In [275]:
@variables κ R M Δκ ΔR ΔM
b = -1/2 *M*R^2*κ

-0.5M*κ*(R^2)

In [276]:
Δb = findErrorFromSym(b,errorPrefix = "Δ")

sqrt(0.25(M^2)*(R^4)*(Δκ^2) + 0.25(R^4)*(ΔM^2)*(κ^2) + (M^2)*(R^2)*(ΔR^2)*(κ^2))

In [277]:
b = substitute(b,
	Dict([
		κ => k.a,
		R => val(r),
		M => val(m)
	]))
b = b.val

Δb = substitute(Δb, 
	Dict([
		κ => k.a, 
		M => val(m), 
		R => val(r), 
		Δκ => val(Δk), 
		ΔM => err(m), 
		ΔR => err(r)
	])
)
Δb = Δb.val

(b = (b ± Δb)*u"kg * m^2 * s^-1") |> latexify

L"$4.18e-5 ± 4.9e-6 kg m^2 s^-1$"

In [278]:
push!(barr, b)
push!(karr, k.a±Δk)

3-element Vector{Any}:
  -0.081 ± 0.011
 -0.0721 ± 0.0082
 -0.0758 ± 0.0088

## barr

In [279]:
barr |> latexify


L"\begin{equation}
\left[
\begin{array}{c}
4.49e-5 ± 6.3e-6 kg m^2 s^-1 \\
3.97e-5 ± 4.6e-6 kg m^2 s^-1 \\
4.18e-5 ± 4.9e-6 kg m^2 s^-1 \\
\end{array}
\right]
\end{equation}
"

In [280]:
karr |> latexify

L"\begin{equation}
\left[
\begin{array}{c}
-0.081 ± 0.011 \\
-0.0721 ± 0.0082 \\
-0.0758 ± 0.0088 \\
\end{array}
\right]
\end{equation}
"